In [1]:
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!pip install tensorboardX
!pip install --quiet pytorch-lightning>=1.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 14.0 MB/s 


In [3]:
import os
import collections
import numpy as np
import random

import pytorch_lightning as pl
from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning.loggers.csv_logs import CSVLogger

%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd

import sys
from tensorboardX import SummaryWriter
import torch
import torch.nn as nn
import torch.utils.data as data
from torch.optim.lr_scheduler import ExponentialLR
import numpy as np
import time
import shutil
import time
import datetime
import argparse
import os
import torch.nn.init as init
import torch.nn.functional as F
from math import floor
from math import ceil
import math

In [4]:
os.chdir('/content/gdrive/MyDrive/gaze_project')
os.getcwd()

'/content/gdrive/MyDrive/gaze_project'

In [5]:
# create dataset
class MyDataset(data.Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __getitem__(self, index):
        feature, target = self.features[index], self.labels[index]
        return feature, target
    
    def __len__(self):
        return len(self.features)
    
# load data.    
def LoadData(dataset_dir, batch_size):

    print("\nLoading the training dataset")
    trainingX = torch.from_numpy(np.load(dataset_dir + 'trainingX.npy')).float()
    trainingY = torch.from_numpy(np.load(dataset_dir + 'trainingY.npy')).float()    
    print('\nTraining Data Size: {}'.format(list(trainingX.size())))

    train_dataset = MyDataset(trainingX, trainingY)

    train_loader = data.DataLoader(dataset=train_dataset, num_workers=8, batch_size=batch_size, shuffle=True)

    print("\nLoading the testing dataset")
    testX = torch.from_numpy(np.load(dataset_dir + 'testX.npy')).float()
    testY = torch.from_numpy(np.load(dataset_dir + 'testY.npy')).float()

    test_size = testX.size()
    test_new_size = test_size[0]//2

    test_X = testX[0:test_new_size, :]
    test_Y = testY[0:test_new_size, :]

    print('\nTest Data Size: {}'.format(list(test_X.size())))
    test_dataset = MyDataset(test_X, test_Y)

    test_loader = data.DataLoader(dataset=test_dataset, num_workers=8, batch_size=batch_size, shuffle=False)

    print("\nLoading the validation data...")
    validationX = testX[test_new_size:, :]
    validationY = testY[test_new_size:, :]

    print('\nValidation Data Size: {}'.format(list(validationX.size())))
    validation_dataset = MyDataset(validationX, validationY)

    validation_loader = data.DataLoader(dataset=validation_dataset, num_workers=8, batch_size=batch_size, shuffle=False)

    return train_loader, test_loader, validation_loader

In [6]:
class HeadObjLSTM(pl.LightningModule):
    def __init__(self, 
                 input_size, 
                 seq_length, 
                 seq_feature_num, 
                 batch_size, 
                 n_output, 
                 dropout_rate,
                 lr,
                 criterion,
                 gamma):
        super().__init__()
        
        # model params
        self.input_size = input_size
        self.seq_feature_num = seq_feature_num
        self.n_layers = 3
        self.hidden_size = 32

        self.n_output = n_output
        self.seq_length = seq_length
        self.seq_feature_num = seq_feature_num
        self.batch_size = batch_size
        self.n_output = n_output
        self.dropout_rate = dropout_rate
        self.lr = lr
        self.criterion = criterion
        self.gamma = gamma
        self.fc_size = 32

        self.lr_scheduler_step = 1

        self.seq_size = self.seq_length * self.seq_feature_num

        self.lstm = nn.LSTM(input_size=self.seq_feature_num,
                            hidden_size = self.hidden_size,
                            num_layers=self.n_layers,
                            dropout=self.dropout_rate,
                            batch_first=True)
        
        self.prd_fc = nn.Sequential(
            nn.Linear(self.hidden_size, self.fc_size),
            nn.BatchNorm1d(self.fc_size),
            nn.ReLU(),
            nn.Dropout(p = self.dropout_rate),
            nn.Linear(self.fc_size, self.fc_size),
            nn.BatchNorm1d(self.fc_size),
            nn.ReLU(),
            nn.Dropout(p = self.dropout_rate),
            nn.Linear(self.fc_size, n_output)
          ) 
        
        
    def forward(self, src):

        headObjectSeq = src.reshape(-1, self.seq_length, self.seq_feature_num)

        output, _ = self.lstm(headObjectSeq)

        output = output[:, -1, :]

        output = self.prd_fc(output)
        return output
    
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=self.lr)
        scheduler = ExponentialLR(optimizer, gamma=self.gamma)
        return [optimizer], [scheduler]

    def training_step(self, batch, batch_idx):
        features, labels = batch

        features = features.reshape(-1, self.input_size).to(device)
        labels = labels.reshape(-1, self.n_output).to(device)

        features = features[:, 0:self.seq_size]

        output = self(features)

        loss = self.criterion(output, labels)

        self.log('train_loss', loss)

        sch = self.lr_schedulers()

        # step every N batches
        if (batch_idx + 1) % self.lr_scheduler_step == 0:
            sch.step()

        # step every N epochs
        if self.trainer.is_last_batch and (self.trainer.current_epoch + 1) % self.lr_scheduler_step == 0:
            sch.step()

        return loss

    def validation_step(self, batch, batch_idx):
        features, labels = batch
        features = features.reshape(-1, self.input_size).to(device)
        labels = labels.reshape(-1, self.n_output).to(device)

        features = features[:, 0:self.seq_size]

        output = self(features)
        loss = self.criterion(output, labels)

        self.log("val_loss", loss, prog_bar=True)
        return loss

    def test_step(self, batch, batch_idx):
        features, labels = batch
        features = features.reshape(-1, self.input_size).to(device)
        labels = labels.reshape(-1, self.n_output).to(device)

        features = features[:, 0:self.seq_size]

        output = self(features)
        loss = self.criterion(output, labels)

        prd_error = 0
        ver_error = 0
        hor_error = 0

        for i in range(output.size(0)):
            prd_error += CalAngularDist(labels[i, 0:2], output[i, 0:2])
            ver_error += abs(labels[i, 0] - output[i, 0])
            hor_error += abs(labels[i, 1] - output[i, 1])

        mean_ver_error = ver_error / output.size(0)
        mean_hor_error = hor_error / output.size(0)
        mean_prd_error = prd_error / output.size(0)

        pixel_pred = AngularCoord2PixelCoord(output[0])
        pixel_gth = AngularCoord2PixelCoord(labels[0])

        prd_x.append(pixel_pred[0])
        prd_y.append(pixel_pred[1])
        gth_x.append(pixel_gth[0])
        gth_y.append(pixel_gth[1])

        self.log("test_loss", loss, prog_bar=True)
        self.log("test_ang_error", mean_prd_error, prog_bar=True)
        self.log("mean_ver_error", mean_ver_error, prog_bar=True)
        self.log("mean_hor_error", mean_hor_error, prog_bar=True)
        return loss

In [7]:
def get_args(train=True):
    args = dict()
    args['feature_num'] = 1702
    args['seq_length'] = 50
    args['seq_feature_num'] = 11
    # the dropout rate of the model.
    args['dropout_rate'] = 0.2  
    args['gradient_clip'] = 0.1  
    # the directory that saves the dataset.
    args['dataset_dir'] = 'DGaze_TrainTest/'
    # the number of total epochs to run
    args['epochs'] = 30
    # the batch size
    args['batch_size'] = 64
    # the initial learning rate.
    args['lr'] = 1e-3
    args['gamma'] = 0.1
    return args

In [8]:

def CalAngularDist(gth, prd):

	vertical_fov = math.pi*110/180;

	screen_w = 1080
	screen_h = 1200
	screen_center_x = 0.5*screen_w
	screen_center_y = 0.5*screen_h

	screen_dist = 0.5* screen_h/math.tan(vertical_fov/2)
	
	gth = AngularCoord2ScreenCoord(gth)
	prd = AngularCoord2ScreenCoord(prd)

	gth[0] = gth[0]*screen_w
	gth[1] = gth[1]*screen_h
	prd[0] = prd[0]*screen_w
	prd[1] = prd[1]*screen_h
	
	#the distance between eye and gth.
	eye2gth = np.sqrt(np.square(screen_dist) + np.square(gth[0] - screen_center_x) + np.square(gth[1] - screen_center_y))
	#the distance between eye and prd.
	eye2prd = np.sqrt(np.square(screen_dist) + np.square(prd[0] - screen_center_x) + np.square(prd[1] - screen_center_y))
	#the distance between gth and prd.
	gth2prd = np.sqrt(np.square(prd[0] - gth[0]) + np.square(prd[1] - gth[1]))
	
	#the angular distance between gth and prd.
	angular_dist = 180/math.pi*math.acos((np.square(eye2gth) + np.square(eye2prd) - np.square(gth2prd))/(2*eye2gth*eye2prd))
	return angular_dist

def AngularCoord2PixelCoord(angular_coord):
	screen_w = 1080
	screen_h = 1200

	screen_coord = AngularCoord2ScreenCoord(angular_coord);

	pixel_coord = np.zeros(2)

	pixel_coord[0] = screen_coord[0]*screen_w
	pixel_coord[1] = screen_coord[1]*screen_h

	return pixel_coord
	
def AngularCoord2ScreenCoord(angular_coord):

	vertical_fov = math.pi*110/180

	screen_w = 1080
	screen_h = 1200

	screen_dist = 0.5* screen_h/math.tan(vertical_fov/2)
	
	screen_coord = np.zeros(2)

	screen_coord[0] = (screen_dist * math.tan(math.pi*angular_coord[0] / 180) + 0.5*screen_w) / screen_w

	screen_coord[1] = (screen_dist * math.tan(-math.pi*angular_coord[1] / 180) + 0.5*screen_h) / screen_h
	return screen_coord

In [9]:
def main(args, train=True):

    # Load dataset
    train_loader, test_loader, validation_loader = LoadData(args['dataset_dir'], args['batch_size'])

    # Create the model.
    print('\n==> Starting...')

    csv_logger = CSVLogger('./', name='head_lstm', version='1'),

    trainer = Trainer(
        max_epochs=args['epochs'],
        logger=csv_logger,
        gpus=1,
        log_every_n_steps=1,
        gradient_clip_val=args['gradient_clip'],
    )

    model = HeadObjLSTM(
        input_size = args['feature_num'],
        seq_length = args['seq_length'], 
        seq_feature_num = args['seq_feature_num'], 
        batch_size = args['batch_size'], 
        n_output = 2, 
        dropout_rate = args['dropout_rate'],
        lr = args['lr'],
        criterion = nn.L1Loss(),
        gamma = args['gamma']
    )

    if train:
      print('\n==> Training...')
      trainer.fit(model, train_dataloaders=train_loader, val_dataloaders=validation_loader)

      metrics = pd.read_csv('./head_lstm/1/metrics.csv')
      train_loss = metrics[['train_loss', 'step', 'epoch']][~np.isnan(metrics['train_loss'])]
      val_loss = metrics[['val_loss', 'epoch']][~np.isnan(metrics['val_loss'])]

      fig, axes = plt.subplots(1, 2, figsize=(16, 5), dpi=100)
      axes[0].set_title('Train loss per batch')
      axes[0].plot(train_loss['step'][::2000], train_loss['train_loss'][::2000])
      axes[1].set_title('Validation loss per epoch')
      axes[1].plot(val_loss['epoch'], val_loss['val_loss'], color='orange')
      plt.show(block = True)

      print(f"Train loss: {train_loss['train_loss'].iloc[-1]:.3f}")
      print(f"Val loss:   {val_loss['val_loss'].iloc[-1]:.3f}")

    else:
      print('\n==> Testing...')
      chk_path = "./head_lstm/0/checkpoints/epoch=11-step=196068.ckpt"
      model2 = model.load_from_checkpoint(chk_path,         
                                          input_size = args['feature_num'],
                                          seq_length = args['seq_length'], 
                                          seq_feature_num = args['seq_feature_num'], 
                                          batch_size = args['batch_size'], 
                                          n_output = 2, 
                                          dropout_rate = args['dropout_rate'],
                                          lr = args['lr'],
                                          criterion = nn.L1Loss(),
                                          gamma = args['gamma'])
      trainer.test(model=model2, dataloaders=test_loader)

In [10]:
def plotComparisonGraph(gth_x, gth_y, prd_x, prd_y):
  s = [5] * len(prd_x)
  plt.figure(figsize=(12,9))

  gth = plt.scatter(gth_x, gth_y, s, color = '#88c999')

  prd = plt.scatter(prd_x, prd_y, s, color = 'hotpink')
  plt.xlim(0, 1080)
  plt.ylim(0, 1200)

  plt.title("Predicted gaze positions versus ground truth", fontsize=16)

  plt.xlabel("Horizontal /pixel", fontsize=16)
  plt.ylabel("Vertical /pixel", fontsize=16)

  plt.legend((prd, gth),
            ('Predicted', 'Ground Truth'),
            scatterpoints=1,
            loc='lower left',
            ncol=3,
            fontsize=12)

  plt.show()

In [11]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print("Device:", device)

# set the random seed to ensure reproducibility
seed_everything(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

train = True

prd_x = []
prd_y = []
gth_x = []
gth_y = []

args = get_args()
main(args, train)

if not train:
  plotComparisonGraph(gth_x, gth_y, prd_x, prd_y)

Global seed set to 42


Device: cuda

Loading the training dataset

Training Data Size: [1045654, 1702]

Loading the testing dataset


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs



Test Data Size: [349152, 1702]

Loading the validation data...

Validation Data Size: [349152, 1702]

==> Starting...

==> Training...


/usr/local/lib/python3.7/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:611: UserWarning: Checkpoint directory ./head_lstm/1/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type       | Params
-----------------------------------------
0 | criterion | L1Loss     | 0     
1 | lstm      | LSTM       | 22.7 K
2 | prd_fc    | Sequential | 2.3 K 
-----------------------------------------
25.0 K    Trainable params
0         Non-trainable params
25.0 K    Total params
0.100     Total estimated model params size (MB)
/usr/local/lib/python3.7/dist-packages/pytorch_lightning/loggers/csv_logs.py:58: UserWarning: Experiment logs directory ./head_lstm/1 exists and is not empty. Previous log files in this directory will be deleted when the new ones are saved!
  f"Experiment logs directory {self.log_dir} exists and is not empty."


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/pytorch_lightning/trainer/trainer.py:726: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


KeyError: ignored